### Save and Retrive Data from Redis
- https://redis.com/

In [1]:
import numpy as np
import pandas as pd
import redis

In [2]:
# Connect to Redis Client
# redis-18692.c57.us-east-1-4.ec2.cloud.redislabs.com:18692
# wCneC8tYJRuw0ovU9Kacbr9kgY4UmQdb
hostname = 'redis-19442.c321.us-east-1-2.ec2.cloud.redislabs.com'
portnumber = 19442
password = 'hT54v2rX7dKEPNaV8s4flSEvmgtF79pn'

r = redis.StrictRedis(host=hostname,
                      port=portnumber,
                      password=password)


In [3]:
r.ping()

True

In [5]:
# r.keys()

#### Load the data (numpy zip)

In [9]:
np_file = np.load('dataframe_students_teacher.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']
# x_values

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Abdul Kalam,Teacher,"[-0.38639936, -1.201683, 0.17475073, -0.233374..."
1,Actor Vijay,Student,"[-0.77745473, -1.0058682, -0.5529729, 0.343021..."
2,Amitabh Bhachchan,Student,"[-0.14923494, 1.0485445, -0.41196242, 0.844359..."
3,Arvind Kejriwal,Student,"[-0.349936, 0.009792837, 0.09995638, -0.312416..."
4,Narendra Modi,Teacher,"[-0.13332304, -0.12751162, -0.07908755, -0.151..."


In [10]:
df['name_role'] = df['Name']+'@'+df['Role']
df

,Name,Role,Facial_Features,name_role
0,Abdul Kalam,Teacher,"[-0.38639936, -1.201683, 0.17475073, -0.233374...",Abdul Kalam@Teacher
1,Actor Vijay,Student,"[-0.77745473, -1.0058682, -0.5529729, 0.343021...",Actor Vijay@Student
2,Amitabh Bhachchan,Student,"[-0.14923494, 1.0485445, -0.41196242, 0.844359...",Amitabh Bhachchan@Student
3,Arvind Kejriwal,Student,"[-0.349936, 0.009792837, 0.09995638, -0.312416...",Arvind Kejriwal@Student
4,Narendra Modi,Teacher,"[-0.13332304, -0.12751162, -0.07908755, -0.151...",Narendra Modi@Teacher


In [25]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)
# records

5

In [26]:
# records[0]

## Save data in Redis
- redis hashes
- keyname = 'academy:register'

In [27]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    
    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    
    # save data into redis cloud
    r.hset(name='academy:register',key=name_role,value=vector_bytes)

In [28]:
vector.dtype

dtype('float32')

### Retrive Records from Redis

In [29]:
retrive_dict = r.hgetall(name='academy:register')
retrive_series = pd.Series(retrive_dict)
retrive_series

b'Abdul Kalam@Teacher'          b'#\xd6\xc5\xbe\xc0\xd0\x99\xbf\xdb\xf12>\xd6\...
b'Arvind Kejriwal@Student'      b'\xd0*\xb3\xbe#r <\xee\xb5\xcc=\x17\xf5\x9f\x...
b'Actor Vijay@Student'          b'F\x07G\xbfJ\xc0\x80\xbf\xa2\x8f\r\xbfs\xa0\x...
b'Amitabh Bhachchan@Student'    b'\x0b\xd1\x18\xbe\xb56\x86?\xbd\xec\xd2\xbe\x...
b'Narendra Modi@Teacher'        b'\xd6\x85\x08\xbeh\x92\x02\xbe\xa7\xf8\xa1\xb...
b'Leanse@Student'               b'\x18\xac\t?\x1fgH\xbf\x03n\x08@sJ\xa7?\xd2;\...
dtype: object

In [30]:
#np.frombuffer(retrive_series[b'Morgan Freeman@Teacher'],dtype=np.float32).shape
retrive_series = retrive_series.apply(lambda x: np.frombuffer(x,dtype=np.float32))
index = retrive_series.index
index = list(map(lambda x: x.decode(), index))

In [31]:
retrive_series.index = index

In [32]:
retrive_df = retrive_series.to_frame().reset_index()
retrive_df.columns = ['name_role','facial_features']
retrive_df

,name_role,facial_features
0,Abdul Kalam@Teacher,"[-0.38639936, -1.201683, 0.17475073, -0.233374..."
1,Arvind Kejriwal@Student,"[-0.349936, 0.009792837, 0.09995638, -0.312416..."
2,Actor Vijay@Student,"[-0.77745473, -1.0058682, -0.5529729, 0.343021..."
3,Amitabh Bhachchan@Student,"[-0.14923494, 1.0485445, -0.41196242, 0.844359..."
4,Narendra Modi@Teacher,"[-0.13332304, -0.12751162, -0.07908755, -0.151..."
5,Leanse@Student,"[0.5377822, -0.7828235, 2.1317146, 1.3069595, ..."


### Registration Form

In [16]:
import cv2
from insightface.app import FaceAnalysis

In [18]:
# configure face analysis
faceapp = FaceAnalysis(name='buffalo_sc',
                       root='insightface_models',
                       providers=['CPUExecutionProvider'])

faceapp.prepare(ctx_id=0, det_size=(640,640), det_thresh=0.5)
# warning: don't set det_thresh < 0.3

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_models\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


##### Step-1: Collect Person and Role

In [19]:
# python input
person_name = input('Enter your name: ')

trials = 3
for i in range(trials):
    role = input("""
    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
    """)
    
    if role in ('1','2'):
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        
        break
    else:
        print('Invalid entry try again')
        if i == 3:
            print('Exceeds maximum trails')
            
key = person_name +'@'+ role
print('Your name = ', person_name)
print('Your role =', role)
print('Key =',key)

Enter your name:  Leanse

    Please choose
    1. Student
    2. Teacher

    Enter number either 1 or 2
     1


Your name =  Leanse
Your role = Student
Key = Leanse@Student


##### Step-2: Collect Samples

In [20]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    
    # get results from insightface model
    results = faceapp.get(frame,max_num=1) # to get only one face
    for res in results:
        sample += 1
        x1, y1, x2, y2 = res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)
        
        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
        
    if sample >= 200: 
        break
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this will trigger only when I press letter q (lowercase q) in my keyboard
        break
        
cap.release()
cv2.destroyAllWindows()

In [21]:
len(face_embeddings)

200

##### Step-3: Optimize Data & Save in Redis Database

In [22]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [23]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [24]:
# save key and facial bytes into redis database
r.hset(name='academy:register',key=key,value=x_mean_bytes)

1